In [1]:
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.core import Dropout
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from keras.models import model_from_json
from keras.layers.pooling import AveragePooling2D
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import argparse
import random
import pickle
import cv2
import os


Using TensorFlow backend.


In [6]:
IMAGE_DIMS = (64, 64, 3)
# initialize the data and labels
data = []
labels = []
face_cascade = cv2.CascadeClassifier('/home/servidorubuntu/opencv-master/data/haarcascades/haarcascade_frontalface_default.xml')

# Convolutional Neural Network
# grab the image paths and randomly shuffle them
print("[INFO] loading images...")
path = '/home/servidorubuntu/Escritorio/Emociones/dataset'
imagePaths = sorted(list(paths.list_images(path)))
print(len(imagePaths))
random.seed(42)
random.shuffle(imagePaths)
print(imagePaths[0:10])

[INFO] loading images...
1610
['/home/servidorubuntu/Escritorio/Emociones/dataset/SAD/03_059.jpg', '/home/servidorubuntu/Escritorio/Emociones/dataset/FEARFUL/04_094.jpg', '/home/servidorubuntu/Escritorio/Emociones/dataset/NEUTRAL/01_189.jpg', '/home/servidorubuntu/Escritorio/Emociones/dataset/HAPPY/02_140.jpg', '/home/servidorubuntu/Escritorio/Emociones/dataset/NEUTRAL/01_001.jpg', '/home/servidorubuntu/Escritorio/Emociones/dataset/FEARFUL/04_069.jpg', '/home/servidorubuntu/Escritorio/Emociones/dataset/DISGUSTED/07_193.jpg', '/home/servidorubuntu/Escritorio/Emociones/dataset/ANGRY/05_173.jpg', '/home/servidorubuntu/Escritorio/Emociones/dataset/NEUTRAL/01_048.jpg', '/home/servidorubuntu/Escritorio/Emociones/dataset/FEARFUL/04_095.jpg']


In [7]:
#loop over the input images
for imagePath in imagePaths:
    # load the image, pre-process it, and store it in the data list
    image = cv2.imread(imagePath)
    face = face_cascade.detectMultiScale(image, 1.1, 7)
    x = face[0][0]
    y = face[0][1]
    w = face[0][2]
    h = face[0][3]
    #print(x,y,h,w)
    crop_img = image[y:y+h, x:x+w]
    img = cv2.resize(crop_img, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
    #img = img_to_array(img)    
    data.append(img)
 
    # extract the class label from the image path and update the
    # labels list
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)

data = np.array(data, dtype="float") / 255
# binarize the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)


In [8]:
print(data[:3])
print(data.shape)
print("labels {}".format(labels))


[[[[0.60392157 0.62745098 0.64705882]
   [0.40392157 0.45098039 0.49019608]
   [0.23921569 0.28627451 0.33333333]
   ...
   [0.28627451 0.36078431 0.38431373]
   [0.54509804 0.57647059 0.57647059]
   [0.61568627 0.63529412 0.63137255]]

  [[0.45098039 0.48627451 0.50196078]
   [0.23921569 0.28235294 0.3372549 ]
   [0.2627451  0.33333333 0.38431373]
   ...
   [0.29019608 0.38039216 0.44313725]
   [0.32156863 0.41176471 0.43921569]
   [0.59607843 0.62352941 0.61176471]]

  [[0.40784314 0.44313725 0.47843137]
   [0.28627451 0.34117647 0.41176471]
   [0.25490196 0.32156863 0.38823529]
   ...
   [0.27058824 0.34509804 0.41176471]
   [0.30196078 0.37254902 0.44313725]
   [0.34901961 0.41960784 0.45098039]]

  ...

  [[0.74901961 0.74509804 0.75294118]
   [0.74117647 0.7372549  0.74509804]
   [0.73333333 0.74117647 0.74509804]
   ...
   [0.69411765 0.69411765 0.69411765]
   [0.69019608 0.69803922 0.69803922]
   [0.67843137 0.68627451 0.68627451]]

  [[0.73333333 0.72941176 0.7372549 ]
   [0.7

In [9]:
print(labels[0:10])
print(labels.shape)
print("Labels {}".format(labels))

[[0 0 0 0 0 1 0]
 [0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0]
 [0 0 1 0 0 0 0]
 [0 1 0 0 0 0 0]
 [1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0]
 [0 0 1 0 0 0 0]]
(1610, 7)
Labels [[0 0 0 ... 0 1 0]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]]


In [10]:
# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data,labels,test_size=0.2,random_state=42)

In [11]:
print("Size {}".format(len(labels)))
print(labels.shape)
print(trainX.shape, testX.shape)
print(trainY.shape, testY.shape)

Size 1610
(1610, 7)
(1288, 64, 64, 3) (322, 64, 64, 3)
(1288, 7) (322, 7)


In [12]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K

def build(width, height, depth, classes):
	# initialize the model along with the input shape to be
	# "channels last" and the channels dimension itself
	model = Sequential()
	inputShape = (height, width, depth)
	chanDim = -1
 
	# if we are using "channels first", update the input shape
	# and channels dimension
	if K.image_data_format() == "channels_first":
		inputShape = (depth, height, width)
		chanDim = 1
	# CONV => RELU => POOL
	model.add(Conv2D(32, (3, 3), padding="same",
		input_shape=inputShape))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=chanDim))
	model.add(MaxPooling2D(pool_size=(3, 3)))
	model.add(Dropout(0.25))
	# (CONV => RELU) * 2 => POOL
	model.add(Conv2D(64, (3, 3), padding="same"))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=chanDim))
	model.add(Conv2D(64, (3, 3), padding="same"))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=chanDim))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.25))
	# (CONV => RELU) * 2 => POOL
	model.add(Conv2D(128, (3, 3), padding="same"))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=chanDim))
	model.add(Conv2D(128, (3, 3), padding="same"))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=chanDim))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.25))
	# first (and only) set of FC => RELU layers
	model.add(Flatten())
	model.add(Dense(1024))
	model.add(Activation("relu"))
	model.add(BatchNormalization())
	model.add(Dropout(0.5))
 
	# softmax classifier
	model.add(Dense(classes))
	model.add(Activation("softmax"))
 
	# return the constructed network architecture
	return model

In [23]:
EPOCHS = 30
INIT_LR = 1e-3
BS = 32


print("[INFO] compiling model...")
model = build(width=IMAGE_DIMS[1], height=IMAGE_DIMS[0],
	depth=IMAGE_DIMS[2], classes=7)

[INFO] compiling model...


In [24]:
# summarize layers
print(model.summary())
# plot graph
plot_model(model, to_file='VGG16Example.png')


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
activation_8 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 64, 64, 32)        128       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 21, 21, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 21, 21, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 21, 21, 64)        18496     
_________________________________________________________________
activation_9 (Activation)    (None, 21, 21, 64)        0         
__________

In [25]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])

In [26]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
	height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
	horizontal_flip=True, fill_mode="nearest")

In [27]:
# train the network
print("[INFO] training network...")
H = model.fit_generator(
	aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY),
	steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS, verbose=1)

[INFO] training network...
Epoch 1/30
40/40 [==============================] - 11s 283ms/step - loss: 2.9499 - acc: 0.2352 - val_loss: 2.6714 - val_acc: 0.3354
Epoch 2/30
40/40 [==============================] - 10s 249ms/step - loss: 2.2155 - acc: 0.3313 - val_loss: 2.5309 - val_acc: 0.2857
Epoch 3/30
40/40 [==============================] - 10s 249ms/step - loss: 1.8236 - acc: 0.3977 - val_loss: 1.5301 - val_acc: 0.4720
Epoch 4/30
40/40 [==============================] - 10s 252ms/step - loss: 1.6644 - acc: 0.4586 - val_loss: 1.3873 - val_acc: 0.5404
Epoch 5/30
40/40 [==============================] - 10s 252ms/step - loss: 1.5465 - acc: 0.4438 - val_loss: 1.2701 - val_acc: 0.6087
Epoch 6/30
40/40 [==============================] - 11s 263ms/step - loss: 1.4478 - acc: 0.4876 - val_loss: 1.1778 - val_acc: 0.5932
Epoch 7/30
40/40 [==============================] - 10s 258ms/step - loss: 1.2544 - acc: 0.5336 - val_loss: 1.2880 - val_acc: 0.5404
Epoch 8/30
40/40 [========================

In [28]:
import matplotlib
matplotlib.use("Agg")
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper left")
plt.savefig("EmoTrain_1.png")
#plt.show()

In [29]:
scores = model.evaluate(trainX, trainY, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 40.53%


In [22]:
from keras.models import model_from_json
# serialize model to JSON
model_json = model.to_json()
with open("EmoClassModel.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("EmoClassWeights.h5")
print("Saved model to disk")

Saved model to disk
